# Modifying an image in a project
In this notebook we'll show how to take a single (annotated) image from a project, and modify it by converting it to grayscale. 

True in-place modification of images in the project is not possible, so we'll take a step by step approach: First we'll download the image, then convert it to grayscale on our local machine and then upload it again. Finally, we'll have to take the annotation from the original image and apply it to the grayscale image we created. 

In [2]:
# As usual we'll connnect to the platform first, and create the project manager

from sc_api_tools import SCRESTClient
from sc_api_tools.rest_managers import ProjectManager

client = SCRESTClient(
    host='https://10.91.242.203/',
    username='sc-api-tools@intel.com',
    password='Inteldemos!'
)
project_manager = ProjectManager(session=client.session, workspace_id=client.workspace_id)

Authenticating on host https://10.91.242.203/...
Authentication successful. Cookie received.


## Getting a project
First, let's get a project to work with. You can either create a new, COCO-based project with a few images and annotations, or select an existing project from the workspace.

#### Option 1: Creating a new project
Execute the next two code cells if you'd like to create a new project

In [3]:
from sc_api_tools.demos import get_coco_dataset

COCO_PATH = get_coco_dataset()

COCO dataset (subset: val2017) found at path c:\users\ljcornel\pycharmprojects\frameworks.ai.interactive-ai-workflow.sonoma-creek-api-tools\data


In [4]:
from sc_api_tools.annotation_readers import DatumAnnotationReader

# Create the annotation reader and select the images with 'horse' in them
annotation_reader = DatumAnnotationReader(base_data_folder=COCO_PATH, annotation_format='coco')
annotation_reader.filter_dataset(labels=['horse'])

# Create the project, upload images and annotations
project = client.create_single_task_project_from_dataset(
    project_name='COCO horse detection demo',
    project_type='detection',
    path_to_images=COCO_PATH,
    annotation_reader=annotation_reader,
    number_of_images_to_upload=12, 
    number_of_images_to_annotate=12,
    enable_auto_train=True
)

Datumaro dataset consisting of 5000 items in coco format was loaded from c:\users\ljcornel\pycharmprojects\frameworks.ai.interactive-ai-workflow.sonoma-creek-api-tools\data
Datumaro dataset was created in 7.3 seconds
After filtering, dataset with labels ['horse'] contains 128 items.
Project created successfully.
Starting image upload...
Upload complete. Uploaded 12 new images in 2.7 seconds.
Annotations have been converted to boxes
Dataset is prepared for detection task.
Starting image annotation upload...
Upload complete. Uploaded 12 new image annotations


#### Option 2: Using an existing project 
If you prefer to use an existing project, the following cell will list all projects in the current workspace so that you can select one

In [5]:
projects = project_manager.list_projects()

7 projects were found on the platform:

 Project: COCO horse detection demo
  Task 1: Detection task
    Labels: ['horse', 'No Object']


 Project: Weeds
  Task 1: Detection task
    Labels: ['pot', 'No Object']
  Task 2: Segmentation task
    Labels: ['clover', 'broadleaf', 'Empty']


 Project: Bottle Anomaly Classification
  Task 1: Anomaly task
    Labels: ['Normal', 'Anomalous']


 Project: Bottle Quality Control
  Task 1: Classification task
    Labels: ['Good', 'Bad']


 Project: Weld classification
  Task 1: Classification task
    Labels: ['Good', 'bad', 'empty']


 Project: COCO animal detection demo
  Task 1: Detection task
    Labels: ['dog', 'horse', 'cat', 'No Object']


 Project: COCO anomalous animal demo
  Task 1: Detection task
    Labels: ['animal', 'No Object']
  Task 2: Anomaly classification task
    Labels: ['Normal', 'Anomalous']




Change the `project_name` here to select your project of choice

In [6]:
project = project_manager.get_project_by_name(project_name="COCO horse detection demo")

## Getting an image 
Now that we have a project we can get an image to modify. Let's get a list of all images in the project, and select the first one in the list as our target image

In [9]:
from sc_api_tools.rest_managers import ImageManager

image_manager = ImageManager(session=client.session, workspace_id=client.workspace_id, project=project)

images = image_manager.get_all_images()
print(f"Project '{project.name}' contains {len(images)} images")

image = images[0]

Project 'COCO horse detection demo' contains 12 images
{'id': '6204ffd7fda2266ee82ecc10',
 'media_information': {'display_url': '/api/v1.0/workspaces/62038997fda2266ee82eb971/projects/6204ffd4fda2266ee82ecc08/datasets/6204ffd4fda2266ee82ecc07/media/images/6204ffd7fda2266ee82ecc10/display/full ',
                       'height': 440,
                       'width': 640},
 'name': '000000016228',
 'state': 'annotated',
 'thumbnail': '/api/v1.0/workspaces/62038997fda2266ee82eb971/projects/6204ffd4fda2266ee82ecc08/datasets/6204ffd4fda2266ee82ecc07/media/images/6204ffd7fda2266ee82ecc10/display/thumb',
 'type': 'image',
 'upload_time': '2022-02-10T12:06:47.483000+00:00'}


## Converting to grayscale
To convert the image to grayscale, we first have to fetch the pixel data from the platform. This can be done using the `image.get_data()` method. This method takes the `client.session` attribute as it's argument, since this is used to connect to the platform.

In the cell below we fetch the image data and convert it to grayscale. At the end, `cv2.imshow` is used to show the resulting image. This will open up a separate window, displaying the image.

In [11]:
import cv2

image_numpy = image.get_data(session=client.session)

# Convert numpy data to grayscale
grayscale_numpy = cv2.cvtColor(image_numpy, cv2.COLOR_RGB2GRAY)

cv2.imshow('Grayscale image', grayscale_numpy)
cv2.waitKey()

-1

#### Uploading the grayscale image
Let's upload the new image to the project

In [13]:
grayscale_image = image_manager.upload_image(image=grayscale_numpy)

# Check that the number of images in the project increases by one when we do this
images = image_manager.get_all_images()
print(f"Project '{project.name}' contains {len(images)} images")

Project 'COCO horse detection demo' contains 13 images


## Annotating the image
Great, we have converted the image to grayscale and uploaded it to the project. However, currently it is still missing an annotation. Since the grayscale conversion of course did not affect the objects in the image, we can just apply the annotation from the original image to the grayscale image directly. 

To do so, we first need to set up an AnnotationManager for the project and retrieve the original annotation

In [24]:
from sc_api_tools.rest_managers import AnnotationManager
from sc_api_tools.utils import show_image_with_annotation_scene

annotation_manager = AnnotationManager(session=client.session, workspace_id=client.workspace_id, project=project)

# Retrieve the annotation for the original image
annotation = annotation_manager.get_annotation(media_item=image)

# Inspect the annotation
print(annotation.overview)
show_image_with_annotation_scene(image, annotation)

{'annotations': [{'labels': [{'color': '#bb4698ff',
                              'name': 'horse',
                              'probability': 1.0,
                              'source': {'id': 'N/A', 'type': 'N/A'}}],
                  'shape': {'height': '0.454',
                            'type': 'RECTANGLE',
                            'width': '0.309',
                            'x': '0.531',
                            'y': '0.348'}}],
 'kind': 'annotation'}


Let's try getting the annotation for the grayscale image. It shouldn't return anything, since we haven't annotated it yet

In [19]:
grayscale_annotation = annotation_manager.get_annotation(media_item=grayscale_image)
print(grayscale_annotation)

None


#### Applying the annotation
Alright, now we're all set to annotate the grayscale image. Note that we passed `media_item=grayscale_image` to the `get_annotation` method in the cell above. It's important to use the `grayscale_image` rather than `grayscale_numpy`, because the grayscale_image holds the actual reference to the image on the platform. grayscale_numpy is just the pixel data, and the annotation_manager wouldn't know how to handle it.

In [20]:
grayscale_annotation = annotation_manager.upload_annotation(media_item=grayscale_image, annotation_scene=annotation)

# Inspect the annotation
show_image_with_annotation_scene(grayscale_image.get_data(client.session), grayscale_annotation)

That's it! To make sure that the annotation was indeed uploaded for the correct image, let's try to get it again using the `get_annotation` method from the annotation_manager

In [26]:
grayscale_annotation = annotation_manager.get_annotation(media_item=grayscale_image)
print(grayscale_annotation.overview)

{'annotations': [{'labels': [{'color': '#bb4698ff',
                              'name': 'horse',
                              'probability': 1.0,
                              'source': {'id': 'N/A', 'type': 'N/A'}}],
                  'shape': {'height': '0.454',
                            'type': 'RECTANGLE',
                            'width': '0.309',
                            'x': '0.531',
                            'y': '0.348'}}],
 'kind': 'annotation'}


## Deleting the original image
This last step is optional: It deletes the RGB image from the project so that only the grayscale variant remains.

In [27]:
image_manager.delete_images([image])

Deleting 1 images from project 'COCO horse detection demo'...
